In [44]:
import pandas as pd 
import wandb
from tqdm import tqdm
import requests
import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib.colors as mcolors

In [45]:
from read_wandb import wandb_results
api = wandb_results("NLP2024_PROJECT_207897091_322720103", wandb_username="noa-levi")

BASE_METRIC = "accuracy_per_mean_user_and_bot"

In [46]:
directory = 'sweeps_csvs'
if not os.path.exists(directory):
    os.makedirs(directory)

In [47]:
df = api.get_sweeps_results(["nmiv2gwu"], metric=BASE_METRIC) 

config_cols = [c for c in df.columns if "config_" in c and c!="config_wandb_run_id" and c!="config_online_simulation_size"]
HPT_cols = [col for col in config_cols if df[col].nunique() > 1 and col != "config_expert_reliability_path"]
print(config_cols)
print(HPT_cols)


Total number of sweeps: 1
Download sweep_id='nmiv2gwu' data...


100%|██████████| 6/6 [00:00<00:00, 22329.92it/s]

['config_seed', 'config_task', 'config_agent', 'config_layers', 'config_dropout', 'config_features', 'config_input_dim', 'config_max_games', 'config_REVIEW_DIM', 'config_hidden_dim', 'config_output_dim', 'config_strategies', 'config_force_train', 'config_ENV_HPT_mode', 'config_architecture', 'config_basic_nature', 'config_total_epochs', 'config_FEATURES_PATH', 'config_bots_per_user', 'config_save_artifacts', 'config_zero_knowledge', 'config_online_sim_type', 'config_use_user_vector', 'config_human_train_size', 'config_loss_weight_type', 'config_ENV_LEARNING_RATE', 'config_personas_balanced', 'config_transformer_nheads', 'config_SIMULATION_EFs_PATH', 'config_OFFLINE_SIM_DATA_PATH', 'config_favorite_topic_method', 'config_personas_group_number', 'config_offline_simulation_size', 'config_simulation_bot_per_user', 'config_simulation_signal_error', 'config_simulation_user_improve', 'config_online_simulation_factor']
['config_seed', 'config_basic_nature']


In [48]:
df[HPT_cols]

,config_seed,config_basic_nature
0,1,30
1,5,29
2,4,29
3,3,29
4,2,29
5,1,29


In [49]:
numeric_cols = df.select_dtypes(include=np.number).columns
df_numeric = df[list(numeric_cols) + [col for col in HPT_cols if col not in numeric_cols]]

grouped = df_numeric.groupby([c for c in HPT_cols if c != "config_seed"])
mean_df = grouped.mean()
std_df = grouped.std()

for col in config_cols:
    if col not in mean_df.columns:
        mean_df[col] = df[col]

best_col = mean_df[[c for c in mean_df.columns if (BASE_METRIC in c and BASE_METRIC[-4:] == c.split("_epoch")[0][-4:])]].idxmax(axis=1)

best_col

config_basic_nature
29    ENV_Test_accuracy_per_mean_user_and_bot_epoch14
30     ENV_Test_accuracy_per_mean_user_and_bot_epoch3
dtype: object

In [50]:
def result_metric(sweeps, group_name, drop_list=[0], drop_HPT=False, metric=BASE_METRIC, epoch="best"):
    df = api.get_sweeps_results(sweeps, metric=metric) 

    config_cols = [c for c in df.columns if "config_" in c and c!="config_wandb_run_id" and c!="config_online_simulation_size"]
    HPT_cols = [col for col in config_cols if df[col].nunique() > 1 and col != "config_expert_reliability_path"]
    print(HPT_cols)
    if drop_HPT:
        df=df.drop([c for c in HPT_cols if not c in ["config_LLM_SIM_SIZE", "config_seed"]], axis=1)
        HPT_cols = ["config_LLM_SIM_SIZE", "config_seed"]
    
    # Remove non-numeric columns before computing mean and std
    numeric_cols = df.select_dtypes(include=np.number).columns
    df_numeric = df[list(numeric_cols) + [col for col in HPT_cols if col not in numeric_cols]]

    grouped = df_numeric.groupby([c for c in HPT_cols if c != "config_seed"])

    mean_df = grouped.mean()
    std_df = grouped.std()
    
    # Re-add non-numeric columns before computing best_col
    for col in config_cols:
        if col not in mean_df.columns:
            mean_df[col] = df[col]

    if epoch=="best":
        best_col = mean_df[[c for c in mean_df.columns if (metric in c and metric[-4:] == c.split("_epoch")[0][-4:])]].idxmax(axis=1)
    else:
        best_col = mean_df[[c for c in mean_df.columns if f"{metric}_epoch{epoch}" in c]].idxmax(axis=1)
    
    result = grouped.apply(lambda x: x[best_col.loc[x.name]].values)
    means = grouped.apply(lambda x: x[best_col.loc[x.name]].mean())
    stds = grouped.apply(lambda x: x[best_col.loc[x.name]].std())


    df_cols = {'mean': means, 'std': stds, 'values': result.values}
    if epoch == "best": df_cols['epoch'] = best_col.apply(lambda x: int(x.split("epoch")[1]) if "epoch" in x else "last")

    df_cols['CI'] = result.apply(lambda x: bootstrap_ci(x))

    summary_df = pd.DataFrame(df_cols, index=best_col.index)

    ##############################################################################################
    summary_df['CI_length'] = summary_df['CI'].apply(lambda x: x[1] - x[0])
    summary_df['Min_Max'] = summary_df['values'].apply(lambda x: (max(x), min(x)))
    ##############################################################################################

    for d in drop_list:
        if d in summary_df.index:
            summary_df=summary_df.drop(d)
    if len(summary_df.index.names) == 1:
        return summary_df.rename_axis(group_name)
    else:
        return summary_df

def bootstrap_ci(data, n_bootstrap=1000, ci=0.95):
    bootstrapped_means = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, size=len(data), replace=True)
        bootstrapped_means.append(np.mean(sample))
    lower_bound = np.percentile(bootstrapped_means, (1 - ci) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_means, (1 + ci) / 2 * 100)
    return lower_bound, upper_bound


# For HyperParameterTuning

For every configuration that you test in the sweep, you will receive in the table the average, standard deviation, all the values obtained for the different seed values, and also the confidence interval within which the result is located at a confidence level of 95%.

When epoch="best" is defined, you can check in which epoch the best result is obtained. If epoch=5 is defined, you will receive the result obtained for epoch number 5.

You can test multiple sweeps simultaneously by entering them into the list found in the first element of the function result_metric.

In [51]:
# Create the directory if it doesn't exist
directory = 'sweeps_csvs'
if not os.path.exists(directory):
    os.makedirs(directory)

HPT_results_df = result_metric(["nmiv2gwu"], "option", drop_HPT=False, epoch="best")
HPT_results_df

Total number of sweeps: 1
Download sweep_id='nmiv2gwu' data...
['config_seed', 'config_basic_nature']


,mean,std,values,epoch,CI,CI_length,Min_Max
option,,,,,,,
29,0.837215,0.001118,"[0.837713383542635, 0.8370784606422689, 0.8363...",14,"(0.8363855996600742, 0.8381450118248992)",0.001759,"(0.8388560459466528, 0.8360910296980735)"
30,0.835826,NaN,[0.8358259748949626],3,"(0.8358259748949626, 0.8358259748949626)",0.000000,"(0.8358259748949626, 0.8358259748949626)"


In [52]:
# sorted(HPT_results_df['mean'], reverse=True)

df_reset = HPT_results_df.copy().reset_index()
#new_df = df_reset[['mean', 'std', 'CI_length']]
new_df = df_reset[['CI_length']]

# new_df = df_reset[['config_basic_nature', 'mean', 'std', 'CI_length']]
#new_df = new_df[new_df['config_basic_nature'].isin([17,18, 19,20,21,22,23,24, 25,26,27,28])]

# new_df.sort_values(by='mean', ignore_index=True, ascending=False)

new_df


,CI_length
0,0.001759
1,0.000000


In [53]:
test_results_df = result_metric(["w4ch5lla"], "option", drop_HPT=False, epoch="best")
test_results_df

Total number of sweeps: 1
Download sweep_id='w4ch5lla' data...


['config_seed', 'config_basic_nature']


,mean,std,values,epoch,CI,CI_length,Min_Max
option,,,,,,,
12,0.836534,0.002257,"[0.835740803191593, 0.8377435333486027, 0.8396...",13,"(0.8349472029566561, 0.838121409601599)",0.003174,"(0.8396895890595983, 0.83375897596534)"
26,0.835456,0.002205,"[0.8329638036610603, 0.83502893224438, 0.83817...",14,"(0.8337413615541263, 0.8371707047985362)",0.003429,"(0.8381719045472741, 0.8329638036610603)"
27,0.834523,0.003230,"[0.8303339650690413, 0.8321916287691351, 0.837...",13,"(0.832015110350452, 0.8370302789978801)",0.005015,"(0.8379674850846894, 0.8303339650690413)"
28,0.835808,0.004232,"[0.8393403951280203, 0.8301136711676507, 0.839...",23,"(0.8325441337532865, 0.8390708924986223)",0.006527,"(0.8399770923838056, 0.8301136711676507)"
32,0.836224,0.002437,"[0.8382810084937229, 0.8359336251264214, 0.832...",19,"(0.8341361503073895, 0.8378079506103159)",0.003672,"(0.8382810084937229, 0.8321569203431703)"
33,0.834661,0.001375,"[0.8348236265585778, 0.8361595805342711, 0.832...",9,"(0.833594695114048, 0.8357179853581599)",0.002123,"(0.8361595805342711, 0.8328257425288769)"


# Result for a specific epoch

In [54]:
sweep_results = result_metric(["w4ch5lla"], "LLMs", drop_HPT=False, epoch=9)
sweep_results

Total number of sweeps: 1
Download sweep_id='w4ch5lla' data...
['config_seed', 'config_basic_nature']


,mean,std,values,CI,CI_length,Min_Max
LLMs,,,,,,
12,0.835512,0.002512,"[0.8364963608632096, 0.8331759866892742, 0.839...","(0.8336704696927028, 0.8375508778247462)",0.003880,"(0.8392864567403172, 0.8331759866892742)"
26,0.832431,0.001480,"[0.8306371314554237, 0.8314481810907247, 0.832...","(0.8313559692366013, 0.8335797675010739)",0.002224,"(0.8344941988264105, 0.8306371314554237)"
27,0.831944,0.000720,"[0.8325437531196987, 0.8309418334128371, 0.831...","(0.8313359028482147, 0.8324578931947995)",0.001122,"(0.8325437531196987, 0.8309418334128371)"
28,0.834684,0.002516,"[0.8341988285624266, 0.8324191205173902, 0.838...","(0.8327572390508405, 0.8369441609876516)",0.004187,"(0.8380754211052169, 0.8323321786854057)"
32,0.834877,0.002224,"[0.8347911920240024, 0.8378227495344027, 0.832...","(0.8331807174801398, 0.8367986793381104)",0.003618,"(0.8378227495344027, 0.8320672964266121)"
33,0.834661,0.001375,"[0.8348236265585778, 0.8361595805342711, 0.832...","(0.8334905650394417, 0.8357179853581599)",0.002227,"(0.8361595805342711, 0.8328257425288769)"
